# Combining Our Local Data Sources

## Combining `.tif` File Data

In [14]:
from rasterio.plot import plotting_extent
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio

In [15]:
# gsa

tif_files = {}
pvo_datasets = {}
for i in range (1, 13):
    if i < 10:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_0{i}.tif"
    else:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_{i}.tif"
    tif_files[i] = tif_file_wst
    
    # Data values
    lats = []
    longs = []
    pvo_vals = []
    
    # Reading the .tif file
    with rasterio.open(tif_file_wst) as ds:
        pvo_data = ds.read(1)  # Reading the first band
        
        # Data dimensions
        h = ds.height
        w = ds.width
        t = ds.transform

        # Extracting the information of each data points from processed .tif file
        for row in range(h):
            for col in range(w):
                pvo_val = pvo_data[row, col]
                if pvo_val != ds.nodata and not np.isnan(pvo_val):
                    
                    # Converting pixel coords to geographic coords
                    long, lat = rasterio.transform.xy(t, row, col)
                    lats.append(lat)
                    longs.append(long)
                    pvo_vals.append(pvo_val)

        # Creating a DataFrame
        df = pd.DataFrame({
            'latitude': lats,
            'longitude': longs,
            'pvo': pvo_vals
        })
        pvo_datasets[i] = df

# Displaying the dataframe for the first .tif file
pvo_datasets[1].head()        

,latitude,longitude,pvo
0,59.979167,-167.179167,0.947
1,59.979167,-167.170833,0.926
2,59.979167,-167.162500,0.905
3,59.979167,-167.154167,0.884
4,59.979167,-167.145833,0.855


In [16]:
# nrel

irradiance_files = ["../Data/nrel/monthly-ghi/ghi_jan.tif", 
                    "../Data/nrel/monthly-ghi/ghi_feb.tif",
                    "../Data/nrel/monthly-ghi/ghi_mar.tif",
                    "../Data/nrel/monthly-ghi/ghi_apr.tif",
                    "../Data/nrel/monthly-ghi/ghi_may.tif",
                    "../Data/nrel/monthly-ghi/ghi_jun.tif",
                    "../Data/nrel/monthly-ghi/ghi_jul.tif",
                    "../Data/nrel/monthly-ghi/ghi_aug.tif",
                    "../Data/nrel/monthly-ghi/ghi_sep.tif",
                    "../Data/nrel/monthly-ghi/ghi_oct.tif",
                    "../Data/nrel/monthly-ghi/ghi_nov.tif",
                    "../Data/nrel/monthly-ghi/ghi_dec.tif"]
irr_datasets = {}
for i in range (1, 13):
    with rasterio.open(irradiance_files[i - 1]) as ds:
        irr_data = ds.read(1) 
        
        height = ds.height
        width = ds.width
        transform = ds.transform
        
        lats, longs, irr_vals = [], [], []
        
        # Extract data points
        for row in range(height):
            for col in range(width):
                irr_val = irr_data[row, col]
                if irr_val != ds.nodata and not np.isnan(irr_val):
                    # Convert pixel coords to geographic coords
                    long, lat = rasterio.transform.xy(transform, row, col)
                    lats.append(lat)
                    longs.append(long)
                    irr_vals.append(irr_val)
        
        irr_df = pd.DataFrame({
            'latitude': lats,
            'longitude': longs,
            'irradiance': irr_vals
        })
        irr_datasets[i] = irr_df

# Displaying the dataframe for the first .tif file
irr_datasets[1].head()

,latitude,longitude,irradiance
0,59.97,-167.58,13.265174
1,59.97,-167.54,13.263229
2,59.97,-167.50,13.261991
3,59.97,-167.46,13.196024
4,59.97,-167.42,13.367926


In [18]:
# Exporting the dataframes to .csv files
for i in range(1, 13):
    if i >= 10:
        pvo_datasets[i].to_csv(f"../Data/csvs/pvout_m{i}.csv", index = False)
        irr_datasets[i].to_csv(f"../Data/csvs/irr_m{i}.csv", index = False)
    else:
        pvo_datasets[i].to_csv(f"../Data/csvs/pvout_m0{i}.csv", index = False)
        irr_datasets[i].to_csv(f"../Data/csvs/irr_m0{i}.csv", index = False)